In [1]:

from google.colab import drive
drive.mount('/content/drive')

!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install transformers[torch] -U

!pip install datasets
!pip install langchain
!pip install langchain_community
!pip install PyMuPDF
!pip install sentence-transformers
!pip install faiss-cpu
!pip install peft

!pip install chromadb
!pip install pypdfium2

import os
import unicodedata
import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments
)

from datasets import Dataset



# 모델 및 토크나이저 설정 (초기)
model_id = "rtzr/ko-gemma-2-9b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# CSV 파일 로드
df_train = pd.read_csv('/content/drive/MyDrive/데이콘/재정정보 AI 검색 알고리즘 경진대회/train.csv')

# 필요한 컬럼 이름으로 데이터셋 준비 (Question과 Answer)
df_train = df_train.rename(columns={"Question": "text", "Answer": "labels"})
dataset = Dataset.from_pandas(df_train)


Mounted at /content/drive
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.5k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [4]:

from google.colab import drive
drive.mount('/content/drive')

!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install transformers[torch] -U

!pip install datasets
!pip install langchain
!pip install langchain_community
!pip install PyMuPDF
!pip install sentence-transformers
!pip install faiss-cpu
!pip install peft

!pip install chromadb
!pip install pypdfium2

import os
import unicodedata
import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments
)
from accelerate import Accelerator
from datasets import Dataset

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

%cd /content/drive/MyDrive/데이콘/재정정보 AI 검색 알고리즘 경진대회

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# 모델 및 토크나이저 설정 (초기)
model_id = "rtzr/ko-gemma-2-9b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# CSV 파일 로드
df_train = pd.read_csv('/content/drive/MyDrive/데이콘/재정정보 AI 검색 알고리즘 경진대회/train.csv')

# 필요한 컬럼 이름으로 데이터셋 준비 (Question과 Answer)
df_train = df_train.rename(columns={"Question": "text", "Answer": "labels"})
dataset = Dataset.from_pandas(df_train)

# 시퀀스 길이를 128로 줄여서 토큰화
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 8비트 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0
)

# 모델 로드 및 8비트 양자화 적용
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation='eager'
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# 모델을 kbit 트레이닝을 위한 준비
model = prepare_model_for_kbit_training(model)

# PEFT (LoRA) 구성
lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=[
      "q_proj",
      "k_proj",
      "v_proj",
      "o_proj",
      "gate_proj",
      "up_proj",
      "down_proj",
      "lm_head",
    ],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

# 모델에 PEFT 적용
model = get_peft_model(model, lora_config)

# 데이터셋의 레이블도 토큰화
def tokenize_labels(examples):
    tokenized_labels = tokenizer(examples['labels'], padding="max_length", truncation=True, max_length=128)
    examples['labels'] = tokenized_labels['input_ids']
    return examples

tokenized_datasets = tokenized_datasets.map(tokenize_labels, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['text'])

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-4,
    per_device_train_batch_size=4,  # 배치 크기 최소화
    per_device_eval_batch_size=1,
    num_train_epochs=40,
    weight_decay=0.001,
    save_total_limit=2,
    logging_steps=5,
    fp16=True,  # Mixed precision training 활성화
    gradient_accumulation_steps=4,  # Gradient Accumulation 사용
    lr_scheduler_type="constant",
    report_to="tensorboard",
)

# Trainer 설정 및 학습 실행ㅎ
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)

# 캐시 정리
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

# Fine-tuning 실행
trainer.train()

# Fine-tuning된 모델 저장
trainer.save_model("./fine_tuned_model")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple/
/content/drive/MyDrive/데이콘/재정정보 AI 검색 알고리즘 경진대회


Map:   0%|          | 0/496 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/852 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00010.safetensors:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Map:   0%|          | 0/496 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
5,6.027500
10,3.056400
15,3.110800
20,2.758000
25,2.864500
30,2.824900
35,2.487600
40,2.642200
45,2.492300
50,2.307900


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 du

In [5]:
from huggingface_hub import login

hugging_face_token = "token" # @param {type:"string"}
# Hugging Face에 로그인
login(token=hugging_face_token)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
model.push_to_hub("Nacho-Cola/gemma2_fine_tuned_model_0817-5_40epoch")
tokenizer.push_to_hub("Nacho-Cola/gemma2_fine_tuned_model_0817-5_40epoch")

adapter_model.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Nacho-Cola/gemma2_fine_tuned_model_0817-5_40epoch/commit/da50db7efdf2cf0a8c55a812dcd9bf1e53cfdb20', commit_message='Upload tokenizer', commit_description='', oid='da50db7efdf2cf0a8c55a812dcd9bf1e53cfdb20', pr_url=None, pr_revision=None, pr_num=None)

In [7]:

import os
import re
from tqdm import tqdm
import chromadb
import argparse
import pandas as pd
import numpy as np
import shutil
from collections import Counter
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
# from get_embedding_function import get_embedding_function
from langchain.vectorstores.chroma import Chroma
from langchain_community.document_loaders import PDFPlumberLoader, PyPDFium2Loader


# Load model directly
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)
import torch


In [8]:
def extract_alphanumeric(input_string: str) -> str:
    # 정규 표현식을 사용하여 숫자와 영문자만 추출
    result = re.findall(r'[a-zA-Z0-9]+', input_string)
    # 리스트를 문자열로 결합하여 반환
    return ''.join(result)

In [9]:
def load_pdf_file(SOURCE_PATH):
  # file_loader = PDFPlumberLoader(SOURCE_PATH)
  file_loader = PyPDFium2Loader(SOURCE_PATH)

  return file_loader.load()

In [10]:
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

In [11]:
from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
Sentence_model = SentenceTransformer('jhgan/ko-sroberta-multitask')
def get_embedding_function(chunks):
  # model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
  # model = SentenceTransformer('jhgan/ko-sroberta-multitask')
  embeddings = []
  for chunk in tqdm(chunks):
    embeddings.append(Sentence_model.encode(chunk))
  return np.array(embeddings)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
def make_prompt(instruction, n_similer):

  PROMPT = f'''당신은 PDF 문서에서 정보를 검색하고 이를 바탕으로 답변하는 AI입니다.
  당신의 역할은 '중앙정부 재정 정보 질의 응답 AI'이고, 사용자의 역할은 '사용자'입니다.
  다양한 재정 관련 텍스트 데이터를 활용해 주어진 질문에 대해 정확도가 높은 답변을 제공해야 합니다.
  답변은 무조건 한문장으로 정리하여 답변하며 질문의 대한 대답을 모두 포함해야 합니다.
  검색 결과만을 사용하여 정보를 검색하여 명확하고 간결한 답변을 제공해야 합니다.
  어떤 상황에서도 한문장으로 답변합니다.
  답변에 한국어만을 사용합니다.
  빈드시 답변에 한국어만을 사용합니다.

  - 검색 결과의 내용을 바탕으로 사용자의 질문에 적합한 답변을 작성합니다.
  - 연도를 포함한 정보는 연도별로 요약하여 설명합니다.
  - 답변은 명확하고 간결하게 작성합니다.
  - 검색 결과의 문장을 최대한 그대로 인용합니다.
  - Answer only in Korean, no matter what.
  ### 검색 결과:
  '''

  instruction = instruction

  # 백터DB 쿼리
  query = instruction

  results = collection.query(
      query_embeddings=Sentence_model.encode([query]),
      n_results=n_similer
  )

  # 결과를 문자열로 변환
  results_content = "\n".join([result["chunk"] for result in results['metadatas'][0]])

  messages = [
      {"role": "system", "content": f"{PROMPT}\n\n\n### 질문:{results_content}"},
      {"role": "user", "content": f"{instruction}"}
  ]

  return messages


In [13]:
LLama3 = None

In [22]:
csv_path = "train.csv" # @param {type:"string"}
llama_model_id = "Nacho-Cola/gemma2_fine_tuned_model_0816" # @param {type:"string"}
similar_n = 5 # @param {type:"integer"}
model_temperature = 0.2 # @param {type:"number"}


login(token=hugging_face_token)

# LLama3 모델을 로드합니다.
if LLama3 is None:
    LLAMA_3_MODEL_ID = llama_model_id

    # 4비트 양자화 설정
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False,
    )

    # 토크나이저 및 모델 로드
    tokenizer = AutoTokenizer.from_pretrained(LLAMA_3_MODEL_ID)
    LLama3 = AutoModelForCausalLM.from_pretrained(
        LLAMA_3_MODEL_ID,
        torch_dtype="auto",
        quantization_config=quantization_config,
        device_map="auto",
    )
    LLama3.eval()  # 모델을 평가 모드로 설정



The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:


# CSV 파일 로드
df = pd.read_csv(csv_path)

# 고유한 PDF 경로 추출
pdfs = df['Source_path'].unique()

# 질문 리스트 초기화
Questions = []
for ind, pdf in enumerate(pdfs):
    Questions.append({
        'index': 'pdf_' + str(ind),
        'Source': pdf,
        'Question': df.loc[df['Source_path'] == pdf]['Question']
    })

answers = []  # 답변 리스트 초기화

# 각 질문에 대해 처리
for Question in tqdm(Questions):
    documents = load_pdf_file(Question['Source'])  # PDF 파일 로드
    split_docs = split_documents(documents)  # 문서 분할

    meta_dict = []  # 메타데이터 저장 리스트
    page_content = []  # 페이지 내용 저장 리스트
    for doc in split_docs:
        meta_dict.append(doc.metadata)  # 메타데이터 추가
        page_content.append(doc.page_content)  # 페이지 내용 추가

    embeddings = get_embedding_function(page_content)  # 임베딩 생성

    chroma_client = chromadb.Client()  # ChromaDB 클라이언트 초기화
    collection = chroma_client.create_collection(extract_alphanumeric(Question['index']+Question['Source']))  # 컬렉션 생성

    metadatas = []  # 메타데이터 저장 리스트
    ids = []  # ID 저장 리스트

    for i, article in tqdm(enumerate(split_docs)):  # 분할된 문서 반복
        metadatas.append({
            'source': article.metadata['source'],  # 소스 추가
            'page': article.metadata['page'],      # 페이지 추가
            'chunk': article.page_content           # 문서 내용 추가
        })
        ids.append(str(i))  # ID 추가

    embeddings_list = embeddings.tolist()  # 임베딩 리스트로 변환

    # ChromaDB에 데이터 추가
    collection.add(
        embeddings=embeddings_list,
        metadatas=metadatas,
        ids=ids
    )

    prompts = []  # 프롬프트 저장 리스트
    for instruction in Question['Question']:
        prompts.append(make_prompt(instruction, n_similer=similar_n))  # 프롬프트 생성

    for message in tqdm(prompts):  # 프롬프트 반복
        input_ids = tokenizer.apply_chat_template(
            message,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(LLama3.device)  # 입력 ID 생성

        input_ids = input_ids.to(torch.long)  # Long 타입으로 변환


        # 모델을 통해 응답 생성
        outputs = LLama3.generate(
            input_ids = input_ids,
            max_new_tokens=128,
            eos_token_id=tokenizer.eos_token_id,
            do_sample=True,
            temperature=model_temperature,
            top_p=0.9,
            repetition_penalty=1.1,
        )

        print('\n'+message[1]['content'])
        response = outputs[0][input_ids.shape[-1]:]  # 응답 추출
        answer = tokenizer.decode(response, skip_special_tokens=True)
        answers.append(answer)  # 응답 디코딩하여 추가

        print(answer)

  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/pypdfium2/_helpers/textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")

100%|██████████| 267/267 [00:04<00:00, 65.46it/s]

267it [00:00, 177984.61it/s]

  2%|▏         | 1/50 [00:05<04:13,  5.17s/it]


2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?
2024년 중앙정부 재정체계는 자본지출, 이전지출, 운영지출로 구성되어 있습니다.



  4%|▍         | 2/50 [00:15<06:23,  7.98s/it]


2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?
2024년 기준, 총지출 656.6조원은 의무지출 347.4조원(52.9%), 재량지출 309.2조원(47.1%)으로 구성되어 있습니다.  




  6%|▌         | 3/50 [00:30<08:55, 11.40s/it]

...


예산 증감률이 가장 높은 분야는 무엇인가요?
2024년 예산 증감률이 가장 높은 분야는 '보건·복지·고용'이며 11.0% 증가했습니다.



 33%|███▎      | 29/88 [06:28<07:24,  7.53s/it]


국방 분야의 예산은 '23년 대비 '24년 어떻게 변했나요?
2024년 국방비는 2023년 대비 4.2% 증가하여 59.4조원입니다.



 34%|███▍      | 30/88 [06:34<06:52,  7.10s/it]


R&D 분야의 예산은 '23년 대비 '24년 어떻게 변했나요?
2024년 R&D 분야 예산은 2023년 대비 △9.5% 감소한 26.5조원입니다.


In [ ]:
def calculate_f1_score(true_sentence, predicted_sentence, sum_mode=True):

    #공백 제거
    true_sentence = ''.join(true_sentence.split())
    predicted_sentence = ''.join(predicted_sentence.split())

    true_counter = Counter(true_sentence)
    predicted_counter = Counter(predicted_sentence)

    #문자가 등장한 개수도 고려
    if sum_mode:
        true_positive = sum((true_counter & predicted_counter).values())
        predicted_positive = sum(predicted_counter.values())
        actual_positive = sum(true_counter.values())

    #문자 자체가 있는 것에 focus를 맞춤
    else:
        true_positive = len((true_counter & predicted_counter).values())
        predicted_positive = len(predicted_counter.values())
        actual_positive = len(true_counter.values())

    #f1 score 계산
    precision = true_positive / predicted_positive if predicted_positive > 0 else 0
    recall = true_positive / actual_positive if actual_positive > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_score

def calculate_average_f1_score(true_sentences, predicted_sentences):

    total_precision = 0
    total_recall = 0
    total_f1_score = 0

    for true_sentence, predicted_sentence in zip(true_sentences, predicted_sentences):
        precision, recall, f1_score = calculate_f1_score(true_sentence, predicted_sentence)
        total_precision += precision
        total_recall += recall
        total_f1_score += f1_score

    avg_precision = total_precision / len(true_sentences)
    avg_recall = total_recall / len(true_sentences)
    avg_f1_score = total_f1_score / len(true_sentences)

    return {
        'average_precision': avg_precision,
        'average_recall': avg_recall,
        'average_f1_score': avg_f1_score
    }


result = calculate_average_f1_score(df['Answer'], answers)
print(result)

In [17]:
df

,SAMPLE_ID,Source,Source_path,Question
0,TEST_000,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
1,TEST_001,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?
2,TEST_002,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?
3,TEST_003,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?
4,TEST_004,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요?
...,...,...,...,...
93,TEST_093,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,재정정책에서 공적보증채무와 다른 일회성 보증은 어떻게 구분되는가?
94,TEST_094,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,미래사회보장급여에 대한 순의무란 무엇을 의미하는가?
95,TEST_095,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,국가결산보고서와 지방자치단체 회계기준에서 우발부채에 대한 용어 및 회계처리가 어떻게...
96,TEST_096,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,"우발부채란 무엇이며, 그 관리가 왜 중요한가?"


In [18]:
submission = pd.read_csv('sample_submission.csv')

In [19]:
submission['Answer'] = answers

In [20]:
submission

,SAMPLE_ID,Answer
0,TEST_000,"2022년 혁신창업사업화자금(융자)의 예산은 2,300,000백만원 입니다."
1,TEST_001,중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 기술력과 사업성이 우수하고 미래...
2,TEST_002,"중소기업진흥에 관한 법률 제66조, 제67조, 제74조와 중소기업창업지원법 제35조..."
3,TEST_003,2010년에는 재창업자금(실패 경영인에 대한 재기지원)이 신규 지원되었습니다.
4,TEST_004,"2020년에는 미래기술육성자금, 고성장촉진자금이 신규 지원되었습니다."
...,...,...
93,TEST_093,공적보증채무는 보증인이 기타 공공부문과 민간부문 단위 기존 채무의 원리금 상환을 보...
94,TEST_094,미래사회보장급여에 대한 순의무는 계약상의 부채는 아니며 미래 발생 비용으로 여겨지는...
95,TEST_095,"국가결산보고서는 우발부채를 재무제표상 부채로 인식하지 않고, 주석에 공시하며 지방자..."
96,TEST_096,"우발부채는 미래에 특정 사건이 발생하면 발생하는 잠재적 의무 또는 현재의무로, 발생..."


In [21]:
submission.to_csv('submission_gemma2_9b_fine_tuned_0817_40epoch.csv', index=False)

In [ ]:
len(answers)

98